# TSLA財報分析

In [17]:
import os
import re
import shutil
import requests
import time
import pandas as pd
from dotenv import load_dotenv
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_core.prompts import ChatPromptTemplate

# RAGAS 相關導入
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy
from datasets import Dataset

# 自動讀取 .env
load_dotenv()

# ==========================================
# 🔑 API KEY 與全域變數設定
# ==========================================
apiKey = os.getenv("OPENAI_API_KEY") or ""
TICKER = "TSLA"
HEADERS = {'User-Agent': "Financial Analyst bot (yourname@company.com)"}

# --- 1. SEC API: 獲取最新財務 Fact 與 Tesla 官網 PDF 連結 ---

def get_latest_sec_data():
    res = requests.get("https://www.sec.gov/files/company_tickers.json", headers=HEADERS)
    cik_data = res.json()
    cik = next(str(v['cik_str']).zfill(10) for v in cik_data.values() if v['ticker'] == TICKER)
    
    sub_url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    sub_data = requests.get(sub_url, headers=HEADERS).json()
    recent = sub_data.get('filings', {}).get('recent', {})
    
    latest_info = {}
    for i, form in enumerate(recent.get('form', [])):
        if form in ['10-K', '10-Q']:
            acc_no_raw = recent['accessionNumber'][i]
            acc_no = acc_no_raw.replace('-', '')
            report_date = recent['reportDate'][i].replace('-', '')
            
            pdf_url = f"https://ir.tesla.com/_flysystem/s3/sec/{acc_no}/tsla-{report_date}-gen.pdf"
            
            latest_info = {
                "ticker": TICKER,
                "form": form,
                "acc_no": acc_no_raw,
                "date": recent['reportDate'][i],
                "url": pdf_url,
                "filename": f"TSLA_{report_date}_{form}.pdf"
            }
            break

    fact_url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    fact_data = requests.get(fact_url, headers=HEADERS).json()
    
    def get_latest_fact(concept):
        try:
            units = fact_data['facts']['us-gaap'][concept]['units']
            unit_key = list(units.keys())[0]
            latest = units[unit_key][-1]
            return {"val": latest['val'], "end": latest['end'], "unit": unit_key}
        except: return {"val": "N/A", "end": "N/A"}

    financials = {
        "Revenue": get_latest_fact("RevenueFromContractWithCustomerExcludingAssessedTax"),
        "NetIncome": get_latest_fact("NetIncomeLoss"),
        "GrossProfit": get_latest_fact("GrossProfit"),
        "EPS": get_latest_fact("EarningsPerShareBasic")
    }
    
    return latest_info, financials

# --- 2. 下載 PDF 並精準提取元素 ---

def download_and_extract_elements(info):
    save_dir = "./reports"
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, info['filename'])
    
    if os.path.exists(save_path):
        print(f"✅ 找到本地 PDF 快取: {info['filename']}，跳過下載與解析。")
        return None 
    
    print(f"📥 正在下載 PDF: {info['url']}")
    res = requests.get(info['url'], headers=HEADERS)
    if res.status_code != 200:
        fallback_url = info['url'].replace("-gen.pdf", ".pdf")
        res = requests.get(fallback_url, headers=HEADERS)
        if res.status_code != 200:
            raise Exception("無法下載 PDF。")
    with open(save_path, 'wb') as f: f.write(res.content)
    
    print(f"📄 正在重新解析 PDF 並處理頁碼元數據: {info['filename']}")
    loader = UnstructuredPDFLoader(save_path, strategy="fast", mode="elements")
    elements = loader.load()
    
    clean_elements = filter_complex_metadata(elements)
    
    docs_with_page = []
    for el in clean_elements:
        text = el.page_content.strip()
        if len(text) > 50:
            docs_with_page.append(el)
            
    return docs_with_page

# --- 3. 建立或讀取向量庫 ---

def get_vector_db(docs, info):
    embeddings = OpenAIEmbeddings(openai_api_key=apiKey)
    persist_dir = f"./chroma_db/{info['ticker']}"
    
    if os.path.exists(persist_dir) and docs is None:
        print(f"📚 直接加載現有向量庫: {persist_dir}")
        return Chroma(persist_directory=persist_dir, embedding_function=embeddings)
    
    print(f"🆕 建立新的向量庫: {persist_dir}")
    if os.path.exists(persist_dir):
        try:
            shutil.rmtree(persist_dir)
            time.sleep(1)
        except:
            persist_dir = f"{persist_dir}_{int(time.time())}"
            
    db = Chroma.from_documents(
        documents=docs, 
        embedding=embeddings, 
        persist_directory=persist_dir
    )
    return db

# --- 4. 檢索除錯函數 ---

def search_with_debug(db, query, k=5):
    """
    執行相似度搜尋並打印每個片段的詳細內容與 PDF 頁碼。
    """
    print(f"\n🔍 搜尋問題: {query}")
    docs = db.similarity_search(query, k=k)
    print(f"🍱 找到 {len(docs)} 個相關片段：")
    
    for i, doc in enumerate(docs):
        page = doc.metadata.get("page_number", "N/A")
        content = doc.page_content.replace('\n', ' ')[:200]
        
        print(f"--- 證據 {i+1} (來源: {TICKER} 財報, 頁碼: {page}) ---")
        print(f"{content}...")
        print("-" * 50)
    
    return docs


# --- 6. 生成報告 ---

def analyze_and_report(db, financials, info):
    llm = ChatOpenAI(model="gpt-4o", openai_api_key=apiKey, temperature=0.3)
    
    search_query = "Detailed analysis of Tesla's operational performance, revenue growth factors, and risk outlook."
    context_docs = db.similarity_search(search_query, k=8)
    
    context_segments = []
    for d in context_docs:
        page = d.metadata.get("page_number", "N/A")
        context_segments.append(f"[來源 PDF 頁碼: {page}]\n{d.page_content}")
    
    context_text = "\n\n".join(context_segments)
    
    prompt = ChatPromptTemplate.from_template("""
    你是一位專業美股研究員。請結合下方的「SEC 官方財務指標」與「財報 PDF 內容」，為 Tesla (TSLA) 撰寫中文分析。

    ### 1. SEC 最新財務數據:
    - 報告類型: {form}, 日期: {date}
    - 總營收: {rev}, 淨利: {ni}, EPS: {eps}

    ### 2. 財報 PDF 檢索內容:
    {context}

    ### 要求:
    - **營運亮點**: 總結本季表現。
    - **深度解析**: 使用數據配合文本，並精確標註 [PDF 第 X 頁]。
    - **風險展望**: 根據 PDF 分析未來風險，並標註 [PDF 第 X 頁]。
    """)
    
    chain = prompt | llm
    report = chain.invoke({
        "form": info['form'], "date": info['date'],
        "rev": financials['Revenue']['val'], "rev_date": financials['Revenue']['end'],
        "ni": financials['NetIncome']['val'], "ni_date": financials['NetIncome']['end'],
        "eps": financials['EPS']['val'],
        "context": context_text
    })
    
    # print("\n📊 專業分析報告已生成。")
    print("\n" + "★"*40)
    print(f"📊 {TICKER} {info['form']} 專業分析報告 (頁碼精確版)")
    print("★"*40)
    print(report.content)
    return report.content

/var/folders/rs/7clwcy050c7gyjz5lqt8kk_00000gn/T/ipykernel_3193/3249585533.py:18: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy
/var/folders/rs/7clwcy050c7gyjz5lqt8kk_00000gn/T/ipykernel_3193/3249585533.py:18: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import faithfulness, answer_relevancy


In [18]:
if __name__ == "__main__":
    try:
        # 流程初始化
         # 1. 檢查 SEC 最新狀態
        info, facts = get_latest_sec_data()
        
        # 2. 檢查 PDF（若已有快取則傳回 None）
        docs = download_and_extract_elements(info)
        
        # 3. 建立或讀取向量庫（若 docs 為 None 則直接加載）
        db = get_vector_db(docs, info)
        
        # 4. 生成報告
        analyze_and_report(db, facts, info)
        
    except Exception as e:
        print(f"❌ 發生錯誤: {e}")

✅ 找到本地 PDF 快取: TSLA_20251231_10-K.pdf，跳過下載與解析。
📚 直接加載現有向量庫: ./chroma_db/TSLA

★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
📊 TSLA 10-K 專業分析報告 (頁碼精確版)
★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
### 營運亮點

在2025財年，Tesla (TSLA) 的總營收達到948.27億美元，淨利潤為37.94億美元，每股盈餘（EPS）為1.18美元。這些數據顯示出Tesla在電動車市場的強勁增長能力，儘管面臨著激烈的市場競爭和不確定的經濟環境，仍然能夠保持穩定的盈利能力。

### 深度解析

Tesla在2025年的財務表現顯示出其在市場中的競爭優勢。總營收的增長顯示出其產品和服務的市場需求持續上升。淨利潤的增長則反映了公司在成本控制和運營效率方面的改善。根據財報內容，Tesla的產品、業務和經營成果受到多方評論，這些評論可能包括批評，並可能對品牌和業務造成負面影響 [PDF 第20頁]。此外，Tesla的成功在很大程度上依賴於Elon Musk的領導，並且公司需要吸引和留住大量的工程、製造、營銷和銷售人才，以支持其高產量產品的銷售和市場擴張 [PDF 第21頁]。

### 風險展望

未來，Tesla面臨多重風險。首先，公司的長期財務穩定性和業務前景受到多方關注，維持市場信心可能具有挑戰性 [PDF 第20頁]。其次，Tesla的未來成功依賴於吸引和留住關鍵人才的能力，尤其是在計劃中的高產量產品銷售和市場擴張中 [PDF 第21頁]。此外，達成運營里程碑的可能性基於對產品路線圖、監管環境、行業趨勢和競爭環境的主觀評估，這些因素可能會影響公司的財務預測 [PDF 第86頁]。因此，Tesla需要在這些方面採取積極措施，以減少對其業務和財務狀況的潛在影響。


# RAGAS評估

In [23]:
def evaluate_retrieval_ragas(db, questions, ground_truths):
    """
    使用 RAGAS 衡量檢索結果。
    """
    print("\n🧪 開始進行 RAGAS 深度評估...")
    llm = ChatOpenAI(model="gpt-4o", openai_api_key=apiKey)
    
    data_samples = {
        "question": [],
        "answer": [],
        "contexts": [],
        "ground_truth": []
    }
    
    for q, gt in zip(questions, ground_truths):
        # 檢索前 3 個片段進行評估
        retrieved_docs = db.similarity_search(q, k=3)
        contexts = [d.page_content for d in retrieved_docs]
        
        # 獲取 LLM 答案以便計算相關性
        prompt = f"請根據以下上下文回答問題: {q}\n\n上下文: {' '.join(contexts)}"
        response = llm.invoke(prompt)
        
        data_samples["question"].append(q)
        data_samples["answer"].append(response.content)
        data_samples["contexts"].append(contexts)
        data_samples["ground_truth"].append(gt)
        
    dataset = Dataset.from_dict(data_samples)
    
    try:
        # 執行評估
        score = evaluate(
            dataset,
            metrics=[
                faithfulness, 
                answer_relevancy, 
                context_precision, 
                context_recall
            ],
            llm=llm
        )
        print("\n✅ 評估完成！")
        return score
    except Exception as e:
        print(f"❌ RAGAS 執行失敗: {e}")
        return None

In [25]:
# 1. 確保最新指標已正確導入 (在 Cell 頂部再次檢查)
from ragas.metrics import (
    faithfulness, 
    answer_relevancy, 
    context_precision, 
    context_recall
)

# 2. 定義測試資料
test_questions = [
    "What are Tesla's primary revenue sources?",
    "What were the key risks mentioned in the latest report?"
]
test_truths = [
    "Tesla generates revenue from automotive sales, energy storage, and services.",
    "Key risks include supply chain disruptions, competition in EV market, and regulatory changes."
]

# 3. 執行評估
# 確保 evaluate_retrieval_ragas 內部使用的是上面導入的對象
result = evaluate_retrieval_ragas(db, test_questions, test_truths)

if result:
    # 將結果轉為 DataFrame
    df = result.to_pandas()
    
    # 定義我們想要觀察的欄位名稱
    # 注意：RAGAS 在 DataFrame 中的欄位名通常與指標對象名稱一致
    target_cols = [
        "question", 
        "context_precision", 
        "context_recall", 
        "faithfulness", 
        "answer_relevancy"
    ]
    
    # 自動過濾掉不存在的欄位 (避免 KeyError)
    existing_cols = [c for c in target_cols if c in df.columns]
    
    print("\n" + "="*60)
    print("📊 RAGAS 深度評估結果報表")
    print("="*60)
    
    # 顯示表格 (Jupyter 特有函數)
    display(df[existing_cols])
    
    # 4. 打印平均分數與解讀建議
    print("\n📈 綜合維度平均分：")
    print("-" * 30)
    for col in existing_cols:
        if col != "question":
            avg_score = df[col].mean()
            # 簡單的解讀邏輯
            status = "✅ 優秀" if avg_score >= 0.8 else "⚠️ 待優化" if avg_score >= 0.5 else "❌ 需大幅調整"
            print(f"{col:18}: {avg_score:.4f}  {status}")
            
    print("\n💡 解讀小撇步：")
    print("- 如果 Context Recall 低：代表檢索器沒抓到 PDF 裡的正確頁面，考慮增加 k 值。")
    print("- 如果 Faithfulness 低：代表 AI 在胡說八道（幻覺），需檢查 Prompt 或 Context 是否太亂。")
    
else:
    print("❌ 無法取得評估結果。請檢查：")
    print("1. OpenAI API Key 是否正確。")
    print("2. 網路是否能連通至 HuggingFace/OpenAI。")
    print("3. 是否已安裝最新版 ragas (pip install ragas -U)。")

/var/folders/rs/7clwcy050c7gyjz5lqt8kk_00000gn/T/ipykernel_3193/2080436868.py:2: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
/var/folders/rs/7clwcy050c7gyjz5lqt8kk_00000gn/T/ipykernel_3193/2080436868.py:2: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (
/var/folders/rs/7clwcy050c7gyjz5lqt8kk_00000gn/T/ipykernel_3193/2080436868.py:2: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import (
/var


🧪 開始進行 RAGAS 深度評估...


Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.



✅ 評估完成！

📊 RAGAS 深度評估結果報表


,context_precision,context_recall,faithfulness,answer_relevancy
0,0.0,0.0,0.583333,0.944241
1,0.0,0.0,1.000000,0.994168



📈 綜合維度平均分：
------------------------------
context_precision : 0.0000  ❌ 需大幅調整
context_recall    : 0.0000  ❌ 需大幅調整
faithfulness      : 0.7917  ⚠️ 待優化
answer_relevancy  : 0.9692  ✅ 優秀

💡 解讀小撇步：
- 如果 Context Recall 低：代表檢索器沒抓到 PDF 裡的正確頁面，考慮增加 k 值。
- 如果 Faithfulness 低：代表 AI 在胡說八道（幻覺），需檢查 Prompt 或 Context 是否太亂。
